Author: Megan Tabbutt

Latest version: 03_25_21

Notes:

Resources: 

- PyPI: https://pypi.org/project/CensusData/
- Documentation: https://jtleider.github.io/censusdata/
- County level data: https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html


Datasets:

    ACS 5-year estimates (2005-2009 to 2015-2019),
    ACS 1-year estimates (2012-2019),
    ACS 3-year estimates (2010-2012 to 2011-2013),
    ACS 1-year supplemental estimates (2014-2019),
    Census 2010 Summary File 1.


## Questions:

_1. What assumptions can we make to not do an $n^2+$ type loop on finding counties?_
- Can we assume it is confined to one state? 
- Will finding NSEW points to check for counties be sufficent? Add a middle point too? 

_2. What resolution do we need for the county level data?_


In [2]:
# Make Jupyter Notebook full screen 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
import censusdata
import pandas as pd
import geopandas
from shapely.geometry import Point
import matplotlib.pyplot as plt

# Feature #1: connect gtfs files to pull counties from CensusData:

In [4]:
# Pull in the latitude and longitude from gtfs data 
gtfs_data_path = "data/mmt_gtfs/"
lat_lon_file = 'shapes.csv'
lat_lon_df = pd.read_csv(gtfs_data_path + lat_lon_file)
lat_lon_df.head(3)

,shape_id,shape_code,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,56620,2F,43.053972,-89.475246,1,0.0000
1,56620,2F,43.053967,-89.474976,2,0.0130
2,56620,2F,43.053933,-89.474855,3,0.0198


### make into a geoDF if you want to plot below to check 
gdf = geopandas.GeoDataFrame(lat_lon_df, geometry=geopandas.points_from_xy(lat_lon_df.shape_pt_lon, lat_lon_df.shape_pt_lat))
gdf.head(3)

In [37]:
# Convert to shapely point objects in tuples
ID_points = [Point(lat_lon_df['shape_pt_lon'][i], lat_lon_df['shape_pt_lat'][i]) for i in range(len(lat_lon_df))]
ID_points[0].coords[:]

[(-89.475246, 43.053971999999995)]

### Trying to reduce the $n^2$ problem

In [38]:
# Find the farthest points to check if multiple counties in the data exist
farthest_points = [0, 0, 0, 0] # North, East, South, West
(farthest_points[0], farthest_points[1], 
 farthest_points[2], farthest_points[3]) = (ID_points[0], ID_points[0], ID_points[0], ID_points[0])

for i in ID_points:
    if i.y > farthest_points[0].y: # north
        farthest_points[0] = i
    if i.x > farthest_points[1].x: # east
        farthest_points[1] = i
    if i.y < farthest_points[2].y: # south
        farthest_points[2] = i
    if i.x < farthest_points[3].x: # west
        farthest_points[3] = i

In [42]:
print(farthest_points[0].x, farthest_points[0].y, "north")
print(farthest_points[1].x, farthest_points[1].y, "east")
print(farthest_points[2].x, farthest_points[2].y, "south")
print(farthest_points[3].x, farthest_points[3].y, "west") # "west? I thought you said w-east..."

-89.25969 43.176925 north
-89.24491 43.176425 east
-89.55833299999999 42.987239 south
-89.564628 42.997069 west


### check that the points are doing the right things... YES
far_points = pd.DataFrame(
    {'Direction': ['North', 'East', 'South', 'West'],
     'Latitude': [farthest_points[0].x, farthest_points[1].x, farthest_points[2].x, farthest_points[3].x],
     'Longitude': [farthest_points[0].y, farthest_points[1].y, farthest_points[2].y, farthest_points[3].y]})

far_points_gdf = geopandas.GeoDataFrame(far_points, geometry=geopandas.points_from_xy(far_points.Longitude, far_points.Latitude))

fig, ax = plt.subplots(1, 1, figsize=(16, 16))
gdf.plot(ax=ax)
far_points_gdf.plot(ax=ax, color='orange', zorder=1)

### Doing the $n^2$ problem: Totally not fesable... on the minutes scale... 

In [55]:
# traverse the whole list? or will it be confined to one state? 
countyData = geopandas.read_file("/Users/megantabbutt/Downloads/cb_2018_us_county_20m/cb_2018_us_county_20m.shp")
countyData.head(3)

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,37,017,01026336,0500000US37017,37017,Bladen,06,2265887723,33010866,"POLYGON ((-78.90200 34.83527, -78.79960 34.850..."
1,37,167,01025844,0500000US37167,37167,Stanly,06,1023370459,25242751,"POLYGON ((-80.49737 35.20210, -80.29542 35.502..."
2,39,153,01074088,0500000US39153,39153,Summit,06,1069181981,18958267,"POLYGON ((-81.68699 41.13596, -81.68495 41.277..."


In [66]:
uniquePoints = ID_points.copy()
counties = []

startingPoint = uniquePoints[0]
startingPoint

for county in countyData.iterrows():
    if county[1]['geometry'].contains(startingPoint):
        startingCounty = county[1]['NAME']
        startingState = county[1]['STATEFP']
        print(startingCounty, " = Starting County")
        print(startingState, " = Starting State")
        
stateDF = countyData[countyData['STATEFP'] == startingState]
countyDF = stateDF[stateDF['NAME'] == startingCounty]

print(len(uniquePoints))

for point in farthest_points:
    for county in countyDF.iterrows():
        if county[1]['geometry'].contains(point):
            if county[1]['NAME'] not in counties:
                counties.append(county[1]['NAME'])
print(counties)

# Need to make this piece fo code work! 
#if len(counties) > 1:
    #for point in uniquePoints:
        #print(point)
        #for county in countyDF.iterrows():
            #if county[1]['geometry'].contains(point):
                #if county[1]['NAME'] not in counties:
                    #counties.append(county[1]['NAME'])
            #uniquePoints.remove(point)
            #print(len(uniquePoints))
            #break
#uniquePoints

Dane  = Starting County
55  = Starting State
114632
['Dane']
